In [2]:
%%capture
%pip install openai==0.27.0

In [3]:
import openai 
import random
import pandas as pd

#determine the model: can be gpt-3.5-turbo or gpt-4
model_name = "gpt-3.5-turbo"

In [2]:
try:
  api_key = "key" # Add API key
  openai.api_key = api_key
  print("API key successfully loaded.")
except:
  print("No API key file found.")

No API key file found.


In [40]:
# CSV path (change accordingly)
csv_path = "lungu_scale_updated_ac.csv" 
prompts = []
fields = ['Instructions', 'Condition', 'Study', 'Type', 'Proposition', 'NAND', 'NOR', 'Positive AND']  # Columns used in CSV
row_order = 1
 
# Reading CSV to dataframe
prompts_df = pd.read_csv(csv_path, usecols=fields)
for _, row in prompts_df.iterrows():
    prompt = {}

    if (row['Condition'] == "Control"):
        text_parts = [row['Instructions'], row['Proposition']]
    
    else:
        # Check if either "NAND" or "NOR" or "AND" exists in the row
        choices = {"NAND":row["NAND"], "NOR":row["NOR"], "Contradictory":row["Positive AND"]}
        available_choices = {key: value for key, value in choices.items() if not pd.isna(value)}

        if available_choices:
            # Randomly choose one from available choices
            chosen_column, content = random.choice(list(available_choices.items()))
        else:
            chosen_column, content = '', ''  # "Control" rows without continuations "A" or "B" or "C"

        text_parts = [row['Instructions'], row['Proposition'], str(content)]
        
    # Filter out empty strings before joining
    prompt["text"] = ' '.join(filter(None, text_parts))
    prompt["condition"] = row['Condition']
    prompt["study"] = row['Study']
    prompt["type"] = row['Type']
    if chosen_column:
        prompt["choice"] = chosen_column
    else:
        prompt["choice"] = "none"
    
    prompt["order"] = row_order
    row_order += 1
    prompts.append(prompt)

random.shuffle(prompts)

print(f"Found {len(prompts)} prompts in the given CSV file.")

Found 131 prompts in the given CSV file.


In [41]:
outputs = []
total_tokens = 0

# Calling chatGPT and appending responses to outputs list
for prompt in prompts:
  response = openai.ChatCompletion.create(
                model=model_name,
                messages=[
                      {"role": "system", "content": "If prompted to respond with a number on a scale, only respond with the whole number. For example, only respond with '6' if you believe it is 6 on the scale. Do not give any explanations. Interpret both sentences together, not separately."},
                      {"role": "user", "content": prompt["text"]},
                  ]
              )
  gpt_answer = response['choices'][0]['message']['content']
  outputs.append(gpt_answer)

  total_tokens += response["usage"]["total_tokens"]

print(f"Messages generated with {total_tokens} tokens")

Messages generated with 14947 tokens


In [42]:
df = pd.DataFrame({"Prompt": [prompt["text"] for prompt in prompts], 
                   "Continuation": [prompt["choice"] for prompt in prompts], 
                   "Condition": [prompt["condition"] for prompt in prompts],
                   "Study": [prompt["study"] for prompt in prompts],
                   "Type": [prompt["type"] for prompt in prompts],
                   "Original order": [prompt["order"] for prompt in prompts],
                   "Response": outputs})

# Save the DataFrame to a CSV file
output_csv_path = "output_responses_5.csv"
df.to_csv(output_csv_path, index=False)

print(f"Responses saved to {output_csv_path}")

Responses saved to output_responses_5.csv


: 